<h1 style="color:#1f77b4; text-align:left; font-size:40px;">
    Model Evaluation
</h1>

<h3 style="color:#555; text-align:left;">
    Test with non used data, Evaluation metrics, Overfitting and Model interpretation
</h3>


<h2 style="color:#1f77b4; border-bottom: 3px solid #1f77b4; padding-bottom:4px;">
</h2>

# **1 Libraries and Data importation**
## **1.1 Import Libraries**

In [ ]:
# --- System utilities ---
import os
import sys
import json
import joblib
from datetime import datetime

# --- Data manipulation ---
import numpy as np
import pandas as pd

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Scikit-learn base utilities ---
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.model_selection import (
    KFold,
    StratifiedKFold,
    cross_val_score,
    cross_validate,
    GridSearchCV,
    RandomizedSearchCV
)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import f1_score

# --- Scikit-learn models ---
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# --- XGBoost ---
from xgboost import XGBClassifier

# --- Optuna ---
import optuna
from optuna.trial import TrialState

## **1.2 Models and Features Subsets Info Loading**

In [3]:
import json
import joblib
import os

artifacts_dir = "../data/processed/Optimized_Model_results"

models_dict = joblib.load(os.path.join(artifacts_dir, "optimized_models.pkl"))

with open(os.path.join(artifacts_dir, "subsets_for_models.json")) as f:
    subsets_for_models = json.load(f)

with open(os.path.join(artifacts_dir, "best_selection_info.json")) as f:
    best_info = json.load(f)

best_model_name = best_info["best_model_name"]
best_subset_features = best_info["best_subset_features"]

print("Artefactos carregados com sucesso!")
print("Modelo vencedor:", best_model_name)
print("Features selecionadas:", len(best_subset_features))


Artefactos carregados com sucesso!
Modelo vencedor: XGB
Features selecionadas: 33


## **1.3 Train and test data Loading**

In [ ]:
## **1.2 Models and Features Subsets Info Loading**

instanciar modelos

In [4]:
xgb_model = models_dict["XGB"]
rf_model = models_dict["RF"]
log_model = models_dict["LOG"]


# 1: Test with non used data

In [ ]:
# 1) Escolher melhor combinação e subset
best_info, best_model_final, X_best = get_best_model_and_data(
    final_models_results,
    subsets_for_models,
    models_dict,
    metric_col="Validation (F1)"
)

# 2) Treinar o modelo FINAL em TODOS os dados (todas as linhas) desse subset
best_model_final.fit(X_best, y.ravel())


In [ ]:
# 1) Escolher melhor combinação e subset
best_info, best_model_final, X_best = get_best_model_and_data(
    final_models_results,
    subsets_for_models,
    models_dict,
    metric_col="Validation (F1)"
)

# 2) Treinar o modelo FINAL em TODOS os dados (todas as linhas) desse subset
best_model_final.fit(X_best, y.ravel())


In [ ]:
y_pred = best_model_final.predict(X_best)
y_proba = best_model_final.predict_proba(X_best)[:, 1]

# 2: Metrics

# 3: Overfitting and Underfitting Evaluation

# 4: Model interpretation

<!-- MODELO: Secção principal numerada -->
<!-- 
<h2 style="background-color:#1f77b4; color:white; padding:10px; border-radius:6px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Secção com linha colorida -->
<!-- 
<h2 style="color:#ff7f0e; border-bottom: 3px solid #ff7f0e; padding-bottom:4px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Subsecção -->
<!-- 
<h3 style="color:#2ca02c; margin-top:10px;">
    X.Y Nome da Subsecção
</h3>
-->

<!-- MODELO: Caixa de Nota -->
<!-- 
<div style="border-left: 5px solid #1f77b4; padding:10px; background-color:#f5f9ff; margin:15px 0;">
    <b>Nota:</b> Texto da nota.
</div>
-->

<!-- MODELO: Secção principal numerada -->
<!-- 
<h2 style="background-color:#1f77b4; color:white; padding:10px; border-radius:6px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Secção com linha colorida -->
<!-- 
<h2 style="color:#ff7f0e; border-bottom: 3px solid #ff7f0e; padding-bottom:4px;">
    X. Nome da Secção
</h2>
-->

<!-- MODELO: Subsecção -->
<!-- 
<h3 style="color:#2ca02c; margin-top:10px;">
    X.Y Nome da Subsecção
</h3>
-->

<!-- MODELO: Caixa de Nota -->
<!-- 
<div style="border-left: 5px solid #1f77b4; padding:10px; background-color:#f5f9ff; margin:15px 0;">
    <b>Nota:</b> Texto da nota.
</div>
-->